In [1]:
import sys
sys.path.insert(0,'../backend')

import webotsgym as wg
from webotsgym.env.reward import WbtRewardGrid
from webotsgym.env import WbtGym
from webotsgym.env.grid.action import WbtActGrid
from webotsgym.env.grid.observation import WbtObsGrid
from webotsgym.config import WbtConfig
from webotsgym.env.reward.steppenalty import step_pen_exp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import gym
from gym import spaces
import stable_baselines
from stable_baselines import A2C, ACER, ACKTR, DQN, DDPG, SAC, PPO1, PPO2, TD3, TRPO
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
config = wg.WbtConfig()

In [2]:
#change here
config.world_size = 8
config.num_obstacles = 16
config.sim_mode = wg.config.SimSpeedMode.RUN

time_steps = 100000
model_name = "PPO_webots_grid"

In [3]:
class MyEval(WbtRewardGrid):
    def __init__(self, env, config: WbtConfig = WbtConfig()):
        super(MyEval, self).__init__(env, config)

    def calc_reward(self):
        if self.env.get_target_distance() < self.targetband:
            reward = 10000
        else:
            reward = 0

            # step penalty
            target_distance = self.env.get_target_distance(normalized=True)
            reward += step_pen_exp(target_distance, step_penalty=-1,
                                   lambda_=5)

            # visited count penalty
            vc = self.env.gps_visited_count
            if vc > 3:
                reward += -0.2 * (vc - 2)**2

            # touching penalty
            if self.env.state.touching is True:
                reward -= 500

        return reward


In [4]:
env = wg.WbtGymGrid(train=True, 
                    config=config,
                    evaluate_class = MyEval)

Accepting on Port:  10201


In [5]:
model = PPO1("MlpPolicy", env)
model.learn(total_timesteps=time_steps)
model.save("model/grid/{}".format(model_name))





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
